##  Extracting Clinical Outcomes w/ Outlines
Utilizing the *outlines* package to support a JSON schema and enforce structure in generated text. By providing a JSON schema of what a therapeutic outcome object should be, can we prompt for outcomes from a provided clinical trials section of an FDA label (faux-RAG).   
   
First, instantiate the model and create a generator object. The generator object can take different kinds of schematics, including multiple choice arrays, or larger dictionary schemas. Examples of each of these taken directly from the cookbook are shown below:

In [1]:
import outlines

model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.3")
generator = outlines.generate.choice(model, ["Blue", "Red", "Yellow"])

color = generator("What is the closest color to Indigo? ")
print(color)
# Blue

/Users/mjc014/.pyenv/versions/3.10.14/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Compiling FSM index for all state transitions: 100%|██████████| 12/12 [00:00<00:00, 12.23it/s]


Blue


In [ ]:
import outlines

# TODO: define outcomes in the $defs, OR look up proper way to get a rtype: [{}] instead of {}
schema = """{
    "$defs": {
        "Armor": {
            "enum": ["leather", "chainmail", "plate"],
            "title": "Armor",
            "type": "string"
        }
    },
    "properties": {
        "name": {"maxLength": 10, "title": "Name", "type": "string"},
        "age": {"title": "Age", "type": "integer"},
        "armor": {"$ref": "#/$defs/Armor"},
        "strength": {"title": "Strength", "type": "integer"}\
    },
    "required": ["name", "age", "armor", "strength"],
    "title": "Character",
    "type": "object"
}"""

model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.3")
generator = outlines.generate.json(model, schema)
character = generator(
    "Generate a new character for my awesome game: "
    + "name, age (between 1 and 99), armor and strength. "
    )
print(character)
# {'name': 'Yuki', 'age': 24, 'armor': 'plate', 'strength': 3}

### Test Run using Outlines
Load FDA label data as search space. 



In [1]:
import pandas as pd

search_space = pd.read_excel('../20240424_trial_searchspace.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)
search_space[0:5]

# TODO: Proper RAG using transformers library?

,brand_name,application_number,adverse_reactions,clinical_studies,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,Dsuvia,NDA209128,6 ADVERSE REACTIONS The following serious adve...,14 CLINICAL STUDIES The efficacy and safety of...,1 INDICATIONS AND USAGE DSUVIA is indicated fo...,4 CONTRAINDICATIONS Use of DSUVIA is contraind...,5 WARNINGS AND PRECAUTIONS Opioid-Induced Hype...,NaN,NaN,12.3 Pharmacokinetics Absorption A single subl...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,WARNING: SERIOUS AND LIFE-THREATENING RISKS FR...,12.2 Pharmacodynamics Effects on the Central N...,NaN
1,ROPINIROLE HYDROCHLORIDE,ANDA090429,6 ADVERSE REACTIONS The following adverse reac...,14 CLINICAL STUDIES 14.1 Parkinson's Disease T...,1 INDICATIONS AND USAGE Ropinirole Tablets are...,4 CONTRAINDICATIONS Ropinirole tablets are con...,5 WARNINGS AND PRECAUTIONS · Sudden onset of s...,NaN,NaN,12.3 Pharmacokinetics Ropinirole displayed lin...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Clinical experience with...,NaN
2,Amlodipine Besylate,ANDA076846,6 ADVERSE REACTIONS Most common adverse reacti...,14 CLINICAL STUDIES 14.1 Effects in Hypertensi...,1 INDICATIONS AND USAGE Amlodipine besylate ta...,4 CONTRAINDICATIONS Amlodipine besylate tablet...,5 WARNINGS AND PRECAUTIONS Symptomatic hypoten...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Hemodynamics: Following ...,NaN
3,Rabeprazole sodium,ANDA202376,6 ADVERSE REACTIONS Most common adverse reacti...,14 CLINICAL STUDIES 14.1 Healing of Erosive or...,1 INDICATIONS AND USAGE Rabeprazole sodium del...,4 CONTRAINDICATIONS Patients with a history of...,5 WARNINGS AND PRECAUTIONS Gastric Malignancy ...,NaN,NaN,12.3 Pharmacokinetics After oral administratio...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Antisecretory Activity T...,NaN
4,Phentermine hydrochloride,ANDA205017,6 ADVERSE REACTIONS The following adverse reac...,14 CLINICAL STUDIES In relatively short-term c...,1 INDICATIONS AND USAGE Phentermine hydrochlor...,4 CONTRAINDICATIONS • History of cardiovascula...,5 WARNINGS AND PRECAUTIONS Coadministration wi...,NaN,NaN,12.3 Pharmacokinetics Following the administra...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Typical actions of amphe...,NaN


Create a JSON schema for a Therapeutic Outcome object. 

In [2]:
schema = """{
    "properties": {
        "outcome metric": {"title": "Outcome Metric", "description": "The name of the primary outcome metric that has been measured.", "type": "string"},
        "measured value": {"title": "Measured Value", "description": "The measured value associated with the outcome metric. It is taken directly from the text" ,"type": "string"},
        "study group": {"title": "Study Group", "description": "The group within the study from which the outcome was measured.", "type": "string"}\
    },
    "required": ["outcome metric", "measured value", "study group"],
                
    "title": "Outcomes",
    "description": "Primary treatment outcomes from a clinical trial. There can be more than one per clinical trial.",
    "type": "object"
}"""
# TODO: Return an array of multiple objects as the schema

Instantiate the model and combine with the therapeutic outcome schema to create a generator object.

In [1]:
# outlines
import outlines
model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.3")
generator = outlines.generate.json(model, schema)

# RUN ONCE or too much memory usage

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

NameError: name 'schema' is not defined

In [ ]:
# Prompt Garbage
#         "descriptors": {"title": "Additional Descriptors", "description": "Additional text from the study that provides context to the outcome.", "type":"string"}\


    # prompt = f'Read the following clinical studies for {row["brand_name"]}. For each treatment outcome in the text, extract the measured outcome, its value, its unit, and the study group. An example of a measured outcome is "Response Rate", an example value would be "56", and example unit would be "patients", and an example study group would be "patients receiving Imatinib". Text: {row["clinical_studies"]}'

For each label, run a prompt through the generator object providing the brand name of the drug as well as the clinical trial text from the FDA label as the search space. Save the structured output and link it with the original dataframe

In [6]:
start_point = 5

test_run = search_space[start_point:start_point+30].copy()
test_run['outcome_raw'] = None


from tqdm import tqdm

position = start_point
for idx, row in tqdm(test_run.iterrows(), total=test_run.shape[0]):
    prompt = f'Extract all therapeutic outcomes for the drug {row["brand_name"]} from this Clinical Trial context:\n{row["clinical_studies"]}'

    outcome = generator(prompt)

    test_run.at[idx, 'outcome_raw'] = outcome
    position += 1
    print(outcome)


  3%|▎         | 1/30 [1:57:53<56:58:57, 7073.69s/it]

{'outcome metric': 'Hamilton Depression Rating Scale (total score)', 'measured value': 'significantly superior to placebo', 'study group': 'venlafaxine'}


  7%|▋         | 2/30 [4:27:31<63:43:44, 8193.74s/it]

{'outcome metric': 'Pain reduction', 'measured value': 'greater than placebo', 'study group': 'Duloxetine delayed-release capsules in Major Depressive Disorder adult trials (Studies MDD-1, MDD-2, MDD-3, and MDD-4)'}


 10%|█         | 3/30 [6:56:10<63:56:15, 8525.01s/it]

...

{'outcome metric': 'complete clearance', 'measured value': '%', 'study group': 'Imiquimod Cream, 3.75% Imiquimod Cream, 2.5% Vehicle Cream'}


 97%|█████████▋| 29/30 [46:26:49<1:34:00, 5640.15s/it]

{'outcome metric': 'safety', 'measured value': 'extracted to 11.5% over 6-months without constipation', 'study group': 'Tramadol 200mg three times a day'}


100%|██████████| 30/30 [47:30:21<00:00, 5700.73s/it]  

{'outcome metric': 'Blood Pressure', 'measured value': 'Increase', 'study group': 'Ephedrine injection'}


In [9]:
# test_run[0:38].to_excel('20240607_llm-mistral.xlsx')

### Group and save
Break apart each dictionary response and save as a new dataframe with explicit columns for each data point to make downstream evaluation just a little bit easier.

In [4]:
import pandas as pd

results = pd.read_excel('20240607_llm-mistral.xlsx')

print(len(results['outcome_raw']))
results['outcome_raw'][0]

30


"{'outcome metric': 'Hamilton Depression Rating Scale (total score)', 'measured value': 'significantly superior to placebo', 'study group': 'venlafaxine'}"

In [12]:
import ast
outcomes = []
values = []
study_groups = []

for clinical_outcome in results['outcome_raw']:
    convert = ast.literal_eval(clinical_outcome)
    outcomes.append(convert['outcome metric'])
    values.append(convert['measured value'])
    study_groups.append(convert['study group'])

clinical_outcomes_df = pd.DataFrame().assign(outcome=outcomes,value=values,study_group=study_groups)
clinical_outcomes_df

,outcome,value,study_group
0,Hamilton Depression Rating Scale (total score),significantly superior to placebo,venlafaxine
1,Pain reduction,greater than placebo,Duloxetine delayed-release capsules in Major D...
2,Success rate,Proportion of subjects with a-0 (clear) or 1 (...,"Clobetasol propionate shampoo, 0.05%"
3,overall survival,12.1 for pemetrexed for injection plus cisplat...,pemetrexed for injection plus cisplatin is com...
4,incidence of endometritis,27.6% for placebo,a group given placebo in the trial
5,dec_flare_rate,decreased frequency of gout flares,colchicine
6,clinical cure rate,183/186,VABOMERE
7,stool frequency,improvement,mesalamine suppositories (500 mg thrice daily)
8,Additional Therapeutic Outcome,mild to moderate hypertension,Lisinopril group
9,Rate of clinical remission,9%,Placebo


In [13]:
# MODIFY
clinical_outcomes_df['brand_name'] = list(results['brand_name'][0:30])
clinical_outcomes_df['application_number'] = list(results['application_number'][0:30])
clinical_outcomes_df['clinical_studies'] = list(results['clinical_studies'][0:30])
clinical_outcomes_df

,outcome,value,study_group,brand_name,application_number,clinical_studies
0,Hamilton Depression Rating Scale (total score),significantly superior to placebo,venlafaxine,venlafaxine,ANDA090555,CLINICAL TRIALS The efficacy of venlafaxine hy...
1,Pain reduction,greater than placebo,Duloxetine delayed-release capsules in Major D...,Duloxetine Delayed-Release,ANDA203088,14 CLINICAL STUDIES 14.1 Overview of the Clini...
2,Success rate,Proportion of subjects with a-0 (clear) or 1 (...,"Clobetasol propionate shampoo, 0.05%",clobetasol propionate,ANDA090974,14 CLINICAL STUDIES The safety and efficacy of...
3,overall survival,12.1 for pemetrexed for injection plus cisplat...,pemetrexed for injection plus cisplatin is com...,Pemetrexed,ANDA204890,14 CLINICAL STUDIES 14.1 Non-Squamous NSCLC In...
4,incidence of endometritis,27.6% for placebo,a group given placebo in the trial,Cefoxitin,ANDA065415,"CLINICAL STUDIES A prospective, randomized, do..."
5,dec_flare_rate,decreased frequency of gout flares,colchicine,Colchicine,ANDA204711,14 CLINICAL STUDIES The evidence for the effic...
6,clinical cure rate,183/186,VABOMERE,VABOMERE,NDA209776,14 CLINICAL STUDIES 14.1 Complicated Urinary T...
7,stool frequency,improvement,mesalamine suppositories (500 mg thrice daily),Mesalamine,ANDA207448,"14 CLINICAL STUDIES Two double-blind, placebo-..."
8,Additional Therapeutic Outcome,mild to moderate hypertension,Lisinopril group,LISINOPRIL,ANDA076180,14 CLINICAL STUDIES 14.1 Hypertension Two dose...
9,Rate of clinical remission,9%,Placebo,Adalimumab,BLA761154,14 CLINICAL STUDIES 14.1 Rheumatoid Arthritis ...


In [14]:
clinical_outcomes_df.to_excel('20240607_llm-mistral.xlsx')